# Homework 3 (Problem 8)
## ME 334: Advanced Dynamics, Control and System Identification
### Stanford University

In [1]:
# imports
import numpy as np
import sympy as sp
from sympy import *
from sympy.vector import *
init_printing(use_unicode=True)

## Setup

In [2]:
# defining free parameters
th1 = symbols(r'\theta_1')  # variables
th2 = symbols(r'\theta_2')
r3 = symbols(r'r_3')
dth1 = symbols(r'\dot{\theta}_1')  # first derivatives
dth2 = symbols(r'\dot{\theta}_2')
dr3 = symbols(r'\dot{r}_3')
ddth1 = symbols(r'\ddot{\theta}_1')  # second derivatives
ddth2 = symbols(r'\ddot{\theta}_2')
ddr3 = symbols(r'\ddot{r}_3')

In [3]:
## helper functions for kinematics
def rot_z(theta):
    R = Matrix([
        [sp.cos(theta), -sp.sin(theta), 0],
        [sp.sin(theta), sp.cos(theta), 0],
        [0, 0, 1],
    ])
    return R

def rot_y(theta):
    R = Matrix([
        [sp.cos(theta), 0, sp.sin(theta)],
        [0,1,0],
        [-sp.sin(theta),0, sp.cos(theta)],
    ])
    return R

def rot_x(theta):
    R = Matrix([
        [1, 0, 0],
        [0, sp.cos(theta), -sp.sin(theta)],
        [0, sp.sin(theta), sp.cos(theta)],
    ])
    return R

In [4]:
## frames and joint variables

## [TODO] Define rotations between frames (you can use the above functions)

R_AB = rot_z(th1)
R_BC = rot_x(th2)

In [5]:
## link geometry
########### (WATCH OUT FOR UNITS!) #############

## [TODO] Fill in the data from the problem

# link 1
H1v = 0.200  #height of vertical cylinder
R1v = 0.025  #radius of vertical cylinder
H1h = 0.300 - R1v  #height (length) of horizontal cylinder
R1h = 0.0175  #radius of horizontal cylinder

# link 2
H2 = 0.250
R2i = 0.006
R2e = 0.012

# link 3
H3 = 0.250
R3 = 0.0055

## density of aluminum
rho = 2700

## kinematic params
L1 = 0.130  #OQ
L2 = H1h  #QR

First, we compute the locations of the centers of mass of each link.

In [20]:
# Helper function.
def mass_cylinder(R, H, rho):
    return np.pi * R ** 2 * H * rho

mv = mass_cylinder(R1v, H1v, rho)
mh = mass_cylinder(R1h, H1h, rho)
m1 = mv + mh
m2 = mass_cylinder(R2e, H2, rho) - mass_cylinder(R2i, H2, rho)
m3 = mass_cylinder(R3, H3, rho)

# Display answer
m1, m2, m3

(1.774656237581747, 0.2290221044466959, 0.06414739499548658)

In [19]:
# location of P1 (COM 1)
mv = np.pi * (R1v ** 2) * H1v * rho  # vertical mass
mh = np.pi * (R1h ** 2) * H1h * rho  # horizontal mass
locv = Matrix([[0], [0], [H1v / 2]])  # vertical location, A frame
loch = Matrix([[-(H1h / 2) * sin(th1)], [(H1h / 2) * cos(th1)], [L1]])  # horizontal location, A frame
A_r_OP1 = (mv * locv + mh * loch) / (mv + mh)

# location of P2 (COM2)
C_r_RP2 = Matrix([[0], [H2/2], [0]])  # location 2, C frame
B_r_QP2 = R_BC * C_r_RP2 + Matrix([[0], [L2], [0]])  # transform to B frame
A_r_OP2 = R_AB * B_r_QP2 + Matrix([[0], [0], [L1]])  # transform to A frame

# location of P3 (COM3)
C_r_RP3 = Matrix([[0], [r3-H3/2], [0]])  # location 2, C frame
B_r_QP3 = R_BC * C_r_RP3 + Matrix([[0], [L2], [0]]) # transform to B frame
A_r_OP3 = R_AB * B_r_QP3 + Matrix([[0], [0], [L1]])  # transform to A frame

# Display answer
A_r_OP3


⎡-((r₃ - 0.125)⋅cos(\theta₂) + 0.275)⋅sin(\theta₁)⎤
⎢                                                 ⎥
⎢((r₃ - 0.125)⋅cos(\theta₂) + 0.275)⋅cos(\theta₁) ⎥
⎢                                                 ⎥
⎣        (r₃ - 0.125)⋅sin(\theta₂) + 0.13         ⎦

In [22]:
# Display answer
N(A_r_OP1.subs([
    (th1, -sqrt(2)),
    (th2, sqrt(3)),
    (r3, sqrt(5)/10),
    (dth1, sqrt(7)),
    (dth2, -sqrt(11)),
    (dr3, sqrt(13)/1000),
]))

⎡0.0546719967680258 ⎤
⎢                   ⎥
⎢0.00863134957303869⎥
⎢                   ⎥
⎣ 0.112076176250934 ⎦

In [23]:
# Display answer
N(A_r_OP2.subs([
    (th1, -sqrt(2)),
    (th2, sqrt(3)),
    (r3, sqrt(5)/10),
    (dth1, sqrt(7)),
    (dth2, -sqrt(11)),
    (dr3, sqrt(13)/1000),
]))

⎡0.251811599996684 ⎤
⎢                  ⎥
⎢0.0397547935799684⎥
⎢                  ⎥
⎣0.253378330623794 ⎦

In [24]:
# Display answer
N(A_r_OP3.subs([
    (th1, -sqrt(2)),
    (th2, sqrt(3)),
    (r3, sqrt(5)/10),
    (dth1, sqrt(7)),
    (dth2, -sqrt(11)),
    (dr3, sqrt(13)/1000),
]))

⎡0.255997358149966 ⎤
⎢                  ⎥
⎢0.0404156207672846⎥
⎢                  ⎥
⎣0.227327536756404 ⎦

Second, we compute the velocities of the centers of mass of each link.

In [15]:
#angular velocities
A_w_AB = dth1 * Matrix([[0], [0], [1]])
A_w_BC = R_AB * (R_BC * (dth2 * Matrix([[1], [0], [0]])))
A_w_AC = A_w_AB + A_w_BC

# computing A_v_AP1 - no translational velocity
A_v_AP1 = A_w_AB.cross(A_r_OP1)  # note: B_w_AB = A_w_AB

# computing A_v_AP2
A_v_AP2 = sp.simplify(A_w_AB.cross(A_r_OP2) + A_w_BC.cross(R_AB * (R_BC * C_r_RP2)))

# computing A_v_AP3
A_v_AP3 = sp.simplify(
    A_w_AB.cross(A_r_OP3)
    + A_w_BC.cross(R_AB * (R_BC * C_r_RP3))
    + R_AB * (R_BC * (dr3 * Matrix([[0], [1], [0]])))
)

In [25]:
# Display answer
N(A_v_AP1.subs([
    (th1, -sqrt(2)),
    (th2, sqrt(3)),
    (r3, sqrt(5)/10),
    (dth1, sqrt(7)),
    (dth2, -sqrt(11)),
    (dr3, sqrt(13)/1000),
]))

⎡-0.0228364044491239⎤
⎢                   ⎥
⎢ 0.144648507127523 ⎥
⎢                   ⎥
⎣         0         ⎦

At the given state, the velocities are:

In [26]:
# Display answer
N(A_v_AP2.subs([
    (th1, -sqrt(2)),
    (th2, sqrt(3)),
    (r3, sqrt(5)/10),
    (dth1, sqrt(7)),
    (dth2, -sqrt(11)),
    (dr3, sqrt(13)/1000),
]))

⎡0.299012162331804⎤
⎢                 ⎥
⎢0.730042973021894⎥
⎢                 ⎥
⎣0.066563224511309⎦

In [27]:
# Display answer
N(A_v_AP3.subs([
    (th1, -sqrt(2)),
    (th2, sqrt(3)),
    (r3, sqrt(5)/10),
    (dth1, sqrt(7)),
    (dth2, -sqrt(11)),
    (dr3, sqrt(13)/1000),
]))

⎡0.211348287520855 ⎤
⎢                  ⎥
⎢0.727553527392018 ⎥
⎢                  ⎥
⎣0.0560674665145466⎦

In [ ]:
# You may use this substitution rule (lengths are in meters here, change if you're doing everything in mm!):
# subs([
#    (th1, -sqrt(2)),
#    (th2, sqrt(3)),
#    (r3, sqrt(5)/10),
#    (dth1, sqrt(7)),
#    (dth2, -sqrt(11)),
#    (dr3, sqrt(13)/1000),
# ])

## Use N() to report values in decimal form
## Specify units of your final answer

Third, we express the angular velocities of each link in convenient frames for the energy calculation.

In [16]:
# Computing these directly from the frames of interest.

# angular velocity of link 1 in frame B
B_w_AB = R_AB.transpose() * A_w_AB  # note: B_w_AB = A_w_AB

# angular velocity of link 2, 3 in frame C
C_w_BC = (R_AB * R_BC).transpose() * A_w_BC
C_w_AC = (R_AB * R_BC).transpose() * A_w_AC

# the angular velocities of links 2 and 3 are identical as they only translate wrt each other

In [29]:
# Display answer
N(B_w_AB.subs([
    (th1, -sqrt(2)),
    (th2, sqrt(3)),
    (r3, sqrt(5)/10),
    (dth1, sqrt(7)),
    (dth2, -sqrt(11)),
    (dr3, sqrt(13)/1000),
]))

⎡       0        ⎤
⎢                ⎥
⎢       0        ⎥
⎢                ⎥
⎣2.64575131106459⎦

In [28]:
# Display answer
N(C_w_AC.subs([
    (th1, -sqrt(2)),
    (th2, sqrt(3)),
    (r3, sqrt(5)/10),
    (dth1, sqrt(7)),
    (dth2, -sqrt(11)),
    (dr3, sqrt(13)/1000),
]))

⎡-3.3166247903554 ⎤
⎢                 ⎥
⎢2.61142704003891 ⎥
⎢                 ⎥
⎣-0.42479267243398⎦

At the given state, the angular velocities are:

In [ ]:
# You may use this substitution rule (lengths are in meters here, change if you're doing everything in mm!):
# subs([
#    (th1, -sqrt(2)),
#    (th2, sqrt(3)),
#    (r3, sqrt(5)/10),
#    (dth1, sqrt(7)),
#    (dth2, -sqrt(11)),
#    (dr3, sqrt(13)/1000),
# ])

## Use N() to report values in decimal form
## Specify units of your final answer

Fourth, we compute the inertia tensors of each link. Following the integration procedure here: https://scienceworld.wolfram.com/physics/MomentofInertiaCylinder.html, we know that the inertia tensor of a solid cylinder is

In [30]:
# helper functions for inertia calculation

def Ic_cylinder(R, H, rho):
    Ic = (np.pi * rho * R ** 2 * H) * Matrix([
        [(H ** 2) / 12 + (R ** 2) / 4, 0, 0],
        [0, (H ** 2) / 12 + (R ** 2) / 4, 0],
        [0, 0, (R ** 2) / 2],
    ])
    return Ic

def inertia_particle(p1, p2):
    x = p1[0] - p2[0]
    y = p1[1] - p2[1]
    z = p1[2] - p2[2]
    I = Matrix([
        [y ** 2 + z ** 2, -x * y, -x * z],
        [-x * y, x ** 2 + z ** 2, -y * z],
        [-x * z, -y * z, x ** 2 + y ** 2],
    ])
    return I

In [31]:
# inertia of link 1 about its center of mass in SRT B
I_hrz = Ic_cylinder(R1h, H1h, rho) # inertia of horizontal segment expressed in a frame with horizontal z axis
I_vert = Ic_cylinder(R1v, H1v, rho) # inertia of vertical segment expressed in a frame with vertical z axis
# as there is axial symmetry, x and y axes are non-unique, any rotation about z doesn't change the inertia tensor
rot_perp = rot_x(np.pi / 2) #rotation of pi/2 about x
I_hrz_rot = rot_perp * I_hrz * rot_perp.transpose()

# shifting theorem to get inertia tensors of both parts about mass center of link 1 in frame B
I_hrz_cen = I_hrz_rot + mh * inertia_particle(loch.subs(th1, 0), A_r_OP1.subs(th1, 0))  #inertia of horizontal segment about mass center of link 1
I_vert_cen = I_vert + mv * inertia_particle(locv.subs(th1, 0), A_r_OP1.subs(th1, 0)) #inertia of vertical segment about mass center of link 1
B_I1 = I_hrz_cen + I_vert_cen

# inertia of link 2 about its center of mass in frame C: this time for the external and internal, point of reference and frame are both already same in the formula
C_I2 = rot_perp.transpose() * (Ic_cylinder(R2e, H2, rho) - Ic_cylinder(R2i, H2, rho)) * rot_perp #originally the z axis was along c2, so we rotate about c1 by -pi/2

#  inertia of link 3 about its center of mass in frame C
C_I3 = rot_perp.transpose() * (Ic_cylinder(R3, H3, rho)) * rot_perp

Finally, we compute the total kinetic energy of the system.

Note that inertias and angular velocities can be expressed in any SRT, but the inertia tensor for any particular link and its angular velocity have to expressed in the same SRT for the energy term to make sense.

In [32]:
T_mat_form = (1 / 2) * (
    m1 * A_v_AP1.transpose() * A_v_AP1
    + m2 * A_v_AP2.transpose() * A_v_AP2
    + m3 * A_v_AP3.transpose() * A_v_AP3
    + B_w_AB.transpose() * B_I1 * B_w_AB
    + C_w_BC.transpose() * (C_I2 + C_I3) * C_w_BC
)
T = sp.simplify(T_mat_form[0])

# Display answer
T

                                2   2    2                                    
0.0320736974977433⋅\dot{\theta}₁ ⋅r₃ ⋅cos (\theta₂) - 0.00801842437443582⋅\dot

         2       2                                            2               
{\theta}₁ ⋅r₃⋅cos (\theta₂) + 0.0176405336237588⋅\dot{\theta}₁ ⋅r₃⋅cos(\theta₂

                                     2    2                                   
) + 0.00229038671439205⋅\dot{\theta}₁ ⋅cos (\theta₂) + 0.00566756813738532⋅\do

          2                                                2                  
t{\theta}₁ ⋅cos(\theta₂) + 0.0202825109206569⋅\dot{\theta}₁  + 0.0320736974977

                 2   2                                    2                   
433⋅\dot{\theta}₂ ⋅r₃  - 0.00801842437443582⋅\dot{\theta}₂ ⋅r₃ + 0.00305924450

                    2                              2
721011⋅\dot{\theta}₂  + 0.0320736974977433⋅\dot{r}₃ 

At the given state, in Nm, the kinetic energy is:



In [33]:
# Display answer
N(T.subs([(th1, -sqrt(2)), (th2, sqrt(3)), (r3, 1/10*sqrt(5)), (dth1, sqrt(7)), (dth2, -sqrt(11)), (dr3, sqrt(13)/1000)]))

0.163123629656796

In [ ]:
# You may use this substitution rule (lengths are in meters here, change if you're doing everything in mm!):
# subs([
#    (th1, -sqrt(2)),
#    (th2, sqrt(3)),
#    (r3, sqrt(5)/10),
#    (dth1, sqrt(7)),
#    (dth2, -sqrt(11)),
#    (dr3, sqrt(13)/1000),
# ])

## Use N() to report values in decimal form
## Specify units of your final answer